In [1]:
using Interact
using RigidBodyDynamics
using RigidBodyTreeInspector

In [2]:
# Open the viewer application
DrakeVisualizer.any_open_windows() || DrakeVisualizer.new_window()

true

In [3]:
# Construct a mechanism
mechanism = rand_chain_mechanism(Float64, [QuaternionFloating{Float64}; [Revolute{Float64} for i = 1:5]]...)

Spanning tree:
Vertex: world (root)
  Vertex: body1, Edge: joint1
    Vertex: body2, Edge: joint2
      Vertex: body3, Edge: joint3
        Vertex: body4, Edge: joint4
          Vertex: body5, Edge: joint5
            Vertex: body6, Edge: joint6
No non-tree joints.

In [4]:
# Construct the visualizer interface, and get the :robot1 subtree
vis = DrakeVisualizer.Visualizer()[:robot1];

In [5]:
# Load the mechanism's geometry into the visualizer
RigidBodyTreeInspector.setgeometry!(vis, RigidBodyTreeInspector.create_geometry(mechanism))

# We can draw the mechanism at a single state:
state = MechanismState(Float64, mechanism)
RigidBodyTreeInspector.settransform!(vis, state);

In [6]:
# Or we can interactively explore the degrees of freedom of the mechanism.
# Note that the quaternion floating base joint, "joint1", has been given 
# six sliders. The first three correspond to the exponential map representation
# of its rotation, and the next three correspond to its translation. 
RigidBodyTreeInspector.inspect(mechanism; show_inertias=true, randomize_colors=true);

In [7]:
# We can also animate the mechanism, given a time sequence of states.
# States will be linearly interpolated between the knot points.
times = collect(linspace(0, 10, 3))
configurations = Vector{Vector{Float64}}(length(times))
for i = 1:length(times)
    rand_configuration!(state)
    configurations[i] = copy(configuration(state))
end
setgeometry!(vis, create_geometry(mechanism))
animate(vis, mechanism, times, configurations; fps = 60., realtimerate = 1.)

In [8]:
# We can simulate a mechanism from an initial state. We'll use a mechanism
# without a QuaternionFloating joint so that it doesn't just fall straight down.
mechanism = rand_chain_mechanism(Float64, [Revolute{Float64} for i = 1:10]...)
vis = Visualizer()[:robot2]
setgeometry!(vis, create_geometry(mechanism))
state = MechanismState(Float64, mechanism)
settransform!(vis, state)
times, configurations, velocities = simulate(state, 10.; Δt = 0.001);

In [9]:
# And we can animate the result in realtime
# (note that it looks a little strange because the link center of mass locations aren't where you think they are)
animate(vis, mechanism, times, configurations)

In [10]:
# We can also inspect individual frames from the simulation
@manipulate for i in 1:length(times)
    set_configuration!(state, configurations[i])
    set_velocity!(state, velocities[i])
    settransform!(vis, state)
    times[i]
end

Interact.Options{:SelectionSlider,Int64}(24: "input-12" = 5001 Int64 ,"i",5001,"5001",Interact.OptionDict(DataStructures.OrderedDict("1"=>1,"2"=>2,"3"=>3,"4"=>4,"5"=>5,"6"=>6,"7"=>7,"8"=>8,"9"=>9,"10"=>10…),Dict(9775=>"9775",2843=>"2843",4495=>"4495",6928=>"6928",9905=>"9905",6380=>"6380",1316=>"1316",6265=>"6265",5459=>"5459",5842=>"5842"…)),Any[],Any[],true,"horizontal")

In [11]:
# 3D visualization can also be done *while simulating*, using DrakeVisualizerSink in combination with
# the lower level RigidBodyDynamics ODE integration functionality:
using RigidBodyDynamics.OdeIntegrators
result = DynamicsResult(Float64, mechanism)
function damped_dynamics!(vd::AbstractArray, sd::AbstractArray, t, state)
    damping = 2.
    τ = -damping * velocity(state)
    dynamics!(result, state, τ)
    copy!(vd, result.v̇)
    copy!(sd, result.ṡ)
    nothing
end
integrator = MuntheKaasIntegrator(damped_dynamics!, runge_kutta_4(Float64), DrakeVisualizerSink(vis))
integrate(integrator, state, 10., 1e-3, maxRealtimeRate = 1.)